# Расчетная работа по курсу: Теория вероятности и статистика
### Вводим константы по варианту

In [1]:
N = 500         # Размер выборки
MX = 3.9        # Мат. ожидание (mean) - средняя
DX = 4.3        # Дисперсия, если взять корень будет ср. квадратичное отклонение
GAMMA = 0.9     # Доверительная вероятность
ALPHA = 0.025   # Уровень значимости